In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets

## 1.数据加载

在 TensorFlow 中,`keras.datasets`模块提供了常用经典数据集的自动下载、管理、加载与转换功能,并且提供了`tf.data.Dataset`数据集对象,方便实现多线程(Multi-threading)、预处理(Preprocessing)、随机打散(Shuffle)和批训练(Training on Batch)等常用数据集的功能。
对于常用的经典数据集,例如:
* Boston Housing,波士顿房价趋势数据集,用于回归模型训练与测试。
* CIFAR10/100,真实图片数据集,用于图片分类任务。
* MNIST/Fashion_MNIST,手写数字图片数据集,用于图片分类任务。
* IMDB,情感分类任务数据集,用于文本分类任务。

通过`datasets.xxx.load_data()`即可实现经典数据集的自动加载，其中xxx代表具体的数据集名称

TensorFlow会默认将数据缓存在用户目录下的.keras/datasets文件夹,用户不需要关心数据集是如何保存的。如果当前数据集不在缓存中,则会自动从网络下载、解压和加载数据集;如果已经在缓存中,则自动完成加载。例如,自动加载MNIST数据集:

In [2]:
# 加载MNIST数据集
(x,y),(x_test,y_test)=datasets.mnist.load_data()
print(f'x: {x.shape}\ny: {y.shape}\nx_test: {x_test.shape}\ny_test: {y_test.shape}')

x: (60000, 28, 28)
y: (60000,)
x_test: (10000, 28, 28)
y_test: (10000,)


通过load_data()函数会返回相应格式的数据,对于图片数据集MNIST、CIFAR10等,会返回2个tuple:
* 第一个tuple保存了用于训练的数据x和y训练集对象
* 第2个tuple则保存了用于测试的数据x_test和y_test测试集对象,所有的数据都用Numpy数组容器保存。

数据加载进入内存后,需要转换成Dataset对象,才能利用TensorFlow提供的各种便捷功能。
通过`Dataset.from_tensor_slices`可以将训练部分的数据图片x和标签y都转换成Dataset对象:

In [3]:
train_db = tf.data.Dataset.from_tensor_slices((x, y)) # 构建 Dataset 对象
print(train_db)

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>


将数据转换成Dataset对象后,一般需要再添加一系列的数据集标准处理步骤,如随机打散、预处理、按批装载等。

## 2.随机打散

通过`Dataset.shuffle(buffer_size)`工具可以设置Dataset对象随机打散数据之间的顺序, 防止每次训练时数据按固定顺序产生

In [4]:
train_db=train_db.shuffle(buffer_size=10000) # 随机打散样本，不会打乱样本与标签映射关系
print(train_db)

<ShuffleDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>


其中,buffer_size参数指定缓冲池的大小,一般设置为一个较大的常数即可。

## 3.批训练
为了利用显卡的并行计算能力,一般在网络的计算过程中会同时计算多个样本,我们把这种训练方式叫做批训练,其中一个批中样本的数量叫做Batch Size。

为了一次能够从Dataset中产生Batch Size数量的样本,需要设置Dataset为批训练方式,实现如下:

In [5]:
train_db=train_db.batch(128) # 设置批训练，batch size为128
print(train_db)

<BatchDataset shapes: ((None, 28, 28), (None,)), types: (tf.uint8, tf.uint8)>


其中128为Batch Size参数,即一次并行计算128个样本的数据。Batch Size一般根据用户的GPU显存资源来设置,当显存不足时,可以适量减少Batch Size来减少算法的显存使用量。

## 4.预处理
从keras.datasets中加载的数据集的格式大部分情况都不能直接满足模型的输入要求, 因此需要根据用户的逻辑自行实现预处理步骤。
Dataset对象通过提供`map(func)`工具函数,可以非常方便地调用用户自定义的预处理逻辑,它实现在`func`函数里。

例如,下方代码调用名为`nothing`的函数完成每个样本的预处理:

In [6]:
# 预处理函数实现在nothing函数中,传入函数名即可
def nothing(x,y):
    pass
    return x,y
train_db = train_db.map(nothing)
print(train_db)

<MapDataset shapes: ((None, 28, 28), (None,)), types: (tf.uint8, tf.uint8)>


现考虑MNIST数据集，经批次划分后加载的图片x shape为\[b,28,28\]，像素使用0～255整型表示，标签为\[b\]大小的向量。

实际的网络输入，需要将图片数据标准化到\[0,1\]或\[-1,1\]，同时根据网络设置，需要将图片shape调整通用的shape（如打平），对于标签需要变成one-hot格式。

In [7]:
def preprocess(x,y):
    # 调用此函数会自动传入x，y
    # 标准化到0~1
    x=tf.cast(x,dtype=tf.float32)/255.
    x=tf.reshape(x,[-1,28*28]) # 打平
    y=tf.cast(y,dtype=tf.int32) # 转换成整型张量
    y=tf.one_hot(y,depth=10) # 进行one-hot编码
    return x,y
train_db=train_db.map(preprocess)
print(train_db)

<MapDataset shapes: ((None, 784), (None, 10)), types: (tf.float32, tf.float32)>


## 5.循环训练
其实就是处理好数据后的迭代方式，迭代训练
又如下几种方式
```python
for step,(x,y) in enumerate(train_db):
    do(x,y)
# or
for x,y in train_db:
    do(x,y)
# generally
for epoch in range(epochs):
    for step,(x,y) in enumerate(train_db):
        do(x,y)
```
另外可以设置Dataset对象，被迭代多少次推出

In [8]:
#下述代码使得for x,y in train_db循环迭代20个epoch才会退出。
train_db=train_db.repeat(20)

(128, 784)
tf.Tensor(
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 10), dtype=float32)
(128, 784)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 10), dtype=float32)
(128, 784)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 10), dtype=float32)
(128, 784)
tf.Tensor(
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]], shape=(128, 10), dtype=float32)
(128, 784)
tf.Tensor(
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0.

In [ ]:
import os
pid=os.getpid()
!kill -9 $pid